# Librerías

In [111]:
# importo librerías

import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [112]:
matrix_df = pd.read_csv('fake_users_1000_users.csv', sep= ',')
matrix_df

,Unnamed: 0,users_id,users_name,users_last_name,users_last_name_2,users_initial,users_years,users_gender,users_sector,users_ccaa,...,viajero,videojuegos,vino,vivo,voluntariado,western,yeye,yoga,zarzuela,zoologia
0,0,1,Helma,Bolnbach,NaN,H. B.,72,mujer,enseñanza,Principado de Asturias,...,0,0,0,0,0,0,0,4,1,0
1,1,2,Gabriel,Oliveira,Morais,G. O.,66,hombre,investigaciontecnica,Canarias,...,2,0,0,0,0,1,0,0,0,0
2,2,3,Walburga,Oestrovsky,NaN,W. O.,70,mujer,amacasa,Comunidad Foral de Navarra,...,1,0,0,0,1,0,2,0,0,1
3,3,4,David,Hernando,Bonet,D. H.,61,hombre,investigaciontecnica,Comunidad de Madrid,...,2,3,0,0,1,0,0,0,0,0
4,4,5,Hassan,Wiek,NaN,H. W.,71,hombre,enseñanza,Cantabria,...,0,2,3,4,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,996,Tilmann,Heydrich,NaN,T. H.,63,hombre,comunicacion,País Vasco,...,0,0,3,0,0,0,1,0,0,0
996,996,997,Josep,Morell,Gibert,J. M.,62,hombre,ingenieria,Región de Murcia,...,0,0,0,2,4,0,5,0,1,0
997,997,998,Feliciana,Cuervo,Piñeiro,F. C.,92,mujer,construccion,Castilla y León,...,1,0,0,0,0,0,3,0,0,0
998,998,999,Ambrosio,Piñol,Manzano,A. P.,60,hombre,seguridad,Comunidad de Madrid,...,0,0,0,0,0,0,0,0,4,4


In [113]:
# elimino columnas
matrix_df.drop('Unnamed: 0', axis = 1, inplace = True)
matrix_df.drop('users_name', axis = 1, inplace = True)
matrix_df.drop('users_last_name', axis = 1, inplace = True)
matrix_df.drop('users_last_name_2', axis = 1, inplace = True)
matrix_df.drop('users_initial', axis = 1, inplace = True)
matrix_df.drop('users_years', axis = 1, inplace = True)
matrix_df.drop('users_gender', axis = 1, inplace = True)
matrix_df.drop('users_sector', axis = 1, inplace = True)
matrix_df.drop('users_ccaa', axis = 1, inplace = True)
matrix_df.drop('users_prov', axis = 1, inplace = True)
matrix_df.drop('users_prox', axis = 1, inplace = True)
matrix_df.drop('users_phone', axis = 1, inplace = True)
matrix_df.drop('users_mail', axis = 1, inplace = True)
matrix_df.drop('users_lang', axis = 1, inplace = True)
matrix_df.drop('users_lgtb', axis = 1, inplace = True)
matrix_df.drop('users_family', axis = 1, inplace = True)
matrix_df.drop('users_relig', axis = 1, inplace = True)
matrix_df.drop('users_politic', axis = 1, inplace = True)
matrix_df.drop('users_vegan', axis = 1, inplace = True)
matrix_df.drop('users_pets', axis = 1, inplace = True)
matrix_df.drop('users_smoke', axis = 1, inplace = True)
matrix_df.drop('users_env', axis = 1, inplace = True)
matrix_df.drop('users_range', axis = 1, inplace = True)
matrix_df.drop('users_budget', axis = 1, inplace = True)
matrix_df.drop('users_sqr_m', axis = 1, inplace = True)
matrix_df.drop('users_bath', axis = 1, inplace = True)
matrix_df.drop('users_outs', axis = 1, inplace = True)
matrix_df.drop('users_facil', axis = 1, inplace = True)
matrix_df.drop('users_drive', axis = 1, inplace = True)
matrix_df.drop('users_max_p', axis = 1, inplace = True)

In [114]:
df= pd.melt(matrix_df, 
            id_vars='users_id', 
            value_vars=list(matrix_df.columns[1:]),
            var_name='Aficiones', 
            value_name='Valoracion')

df

,users_id,Aficiones,Valoracion
0,1,accion,0
1,2,accion,0
2,3,accion,0
3,4,accion,0
4,5,accion,2
...,...,...,...
201995,996,zoologia,0
201996,997,zoologia,0
201997,998,zoologia,0
201998,999,zoologia,4


# Sistema de recomendación ítem-based por ID de usuario
## Modelo colaborativo KNN

In [115]:
# cada fila del df representa un usuario y cada columna representa una AFICIÓN
# hay que crear una matriz de aficiones - usuarios si el sistema de recomendación es ítem-based y una matriz de usuarios - aficiones para revertir
# y rellenar con ceros las celdas donde esa película no haya sido valorada por ningún usuario
# los ceros son necesarios ya que hay que realizar operaciones de álgebra lineal, calculando distancias entre vectores

# las recomendaciones se harán usando KNN con los siguientes input de usuario: usuario, número de usuarios similares en los que basar la recomendación

# entrenaremos el modelo KNN para encontrar usuarios que coincidan en similaridad con el usuario que proporcionamos como input 

In [116]:
# usamos pivot de pandas y creamos matriz aficiones - usuario
matrix_df = df.pivot(
    index='users_id',
     columns='Aficiones',
      values='Valoracion').fillna(0)

matrix_df.head()

Aficiones,accion,activo,ajedrez,alegre,alternativa,amable,animales,antiguedades,apasionado,arte,...,viajero,videojuegos,vino,vivo,voluntariado,western,yeye,yoga,zarzuela,zoologia
users_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,4,0,0,0,...,0,0,0,0,0,0,0,4,1,0
2,0,1,0,0,0,0,0,0,0,3,...,2,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,4,3,0,0,0,...,1,0,0,0,1,0,2,0,0,1
4,0,5,3,0,0,0,2,0,0,0,...,2,3,0,0,1,0,0,0,0,0
5,2,0,2,0,1,0,0,0,4,0,...,0,2,3,4,0,0,0,0,0,1


In [117]:
# transformar matriz en scipy sparse
# se pueden usar en operaciones aritméticasm admiten sumas, restas, multiplicaciones, divisiones y potencia de matrices
# csr_matrix viene de Compressed Sparse Row matrix

matrix_sparse_df = csr_matrix(matrix_df.values)
matrix_sparse_df

<1000x202 sparse matrix of type '<class 'numpy.int64'>'
	with 57281 stored elements in Compressed Sparse Row format>

In [118]:
# entreno el modelo Nearest Neighbours a la scipy sparse matrix

knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(matrix_sparse_df)

NearestNeighbors(algorithm='brute', metric='cosine')

In [119]:
# función para encontrar los top n usuarios más similares al usuario que demos en el input

# determinar ID de usuario y número de usuarios similares que queremos considerar aquí
# por ejemplo en este caso:
# ID del usuario --> 7
# número de usuarios --> 5

def obtener_usuarios_similares(user, n = 5): 
  # input a esta función es el ID del usuario y el número de top usuarios que quieres que el modelo considere
  # número de usuarios se traduce en el valor que asignamos a n aquí
  knn_input = np.asarray([matrix_df.values[user-1]])  
  distances, indices = knn_model.kneighbors(knn_input, n_neighbors=n+1)
  
  print("Los",n,"usuarios más similares al usuario",user, "son: ")
  print(" ")
  for i in range(1,len(distances[0])):
    print(i,". Usuario:", indices[0][i]+1, "separado por una distancia de",distances[0][i])
  return indices.flatten()[1:] + 1, distances.flatten()[1:]


In [122]:
obtener_usuarios_similares(500, 10)

Los 10 usuarios más similares al usuario 500 son: 
 
1 . Usuario: 692 separado por una distancia de 0.5194785007731281
2 . Usuario: 205 separado por una distancia de 0.5433722563328465
3 . Usuario: 313 separado por una distancia de 0.5691453297561011
4 . Usuario: 659 separado por una distancia de 0.5735575232934282
5 . Usuario: 317 separado por una distancia de 0.5758805892219718
6 . Usuario: 826 separado por una distancia de 0.580176704149951
7 . Usuario: 912 separado por una distancia de 0.5806438288743152
8 . Usuario: 871 separado por una distancia de 0.5825759359612317
9 . Usuario: 980 separado por una distancia de 0.5854614241493912
10 . Usuario: 438 separado por una distancia de 0.5880675564894127


(array([692, 205, 313, 659, 317, 826, 912, 871, 980, 438]),
 array([0.5194785 , 0.54337226, 0.56914533, 0.57355752, 0.57588059,
        0.5801767 , 0.58064383, 0.58257594, 0.58546142, 0.58806756]))